In [3]:
word_list = nltk.corpus.words.words()

In [4]:
five_letter_words = []
for word in word_list:
    if len(word) == 5:
        five_letter_words.append(word)


In [5]:
import pandas as pd

word_series = []
char_series = []
for word in five_letter_words:
    word = word.lower()
    word_series.append(word)
    char_series.append([word[0], word[1], word[2], word[3], word[4]])
        

In [6]:
import seaborn as sns

In [7]:
data = pd.DataFrame(data=char_series, index=word_series)

In [8]:
def notIn(location, letter, dataset):
    filtered_dataset = dataset[dataset[location]!=letter]
    return filtered_dataset

def In(location, letter, dataset):
    filtered_dataset = dataset[dataset[location]==letter]
    return filtered_dataset

def InSomewhereElse(location, letter, dataset):
    filtered_dataset = notIn(location, letter, dataset)
    for i in range(5):
        filtered_dataset = pd.concat([filtered_dataset, In(i, letter, filtered_dataset)]).drop_duplicates()
    return filtered_dataset

In [9]:
import nltk
import pandas as pd
import statistics

def solveFirstGuess(data = None):
    if data is None:
        word_list = nltk.corpus.words.words()

        # Get 5 Letter Words
        five_letter_words = []
        word_series = []
        char_series = []
        for word in word_list:
            word = word.lower()
            if len(word) == 5:
                word_series.append(word)
                char_series.append([word[0], word[1], word[2], word[3], word[4]])

        data = pd.DataFrame(char_series, index=word_series)

    prior = 1 / len(data.index)

    p_series = []
    average_series = []
    mean_series = []
    counter = 1
    for guess in data.index:
        print("{} / {} : {}".format(counter, len(data.index), counter/len(data.index)*100))
        p = []
        for answer in data.index:
            filtered_data = data
            for i in range(5):
                m = ''
                if guess[i] == answer[i]:
                    m = 'G'     
                    filtered_data = In(i, guess[i], filtered_data)
                elif guess[i] in answer and guess[i] != answer[i]:
                    m = 'Y'
                    filtered_data = InSomewhereElse(i, guess[i], filtered_data)
                else:
                    m = 'B'
                    filtered_data = notIn(i, guess[i], filtered_data)
            prob = 1 / len(filtered_data.index)
            delta_prob = prob - prior
            p.append(delta_prob)
        counter += 1
        p_series.append(p)
        average_series.append(statistics.mean(p))
        mean_series.append(statistics.median(p))
    data['Score'] = p_series
    data['Average Score'] = average_series
    data['Median Score'] = mean_series
    return data


In [10]:
firstGuess = solveFirstGuess(data)

1 / 10422 : 0.009595087315294569
2 / 10422 : 0.019190174630589137
3 / 10422 : 0.02878526194588371


KeyboardInterrupt: 

1. 6 guesses
2. 5 Spaces with 3 states: 
    1. Not Known
    2. Known
    3. Partially Known 
3. Each guess gives:
    Upto 5 new information.
    1. letters not used
    2. letters used in the correct place
    3. letters used but not in the correct place

Objective: "solve" Wordle

Algorithm: 
Score = delta p
p = answer / possible guesses
